In [58]:
%load_ext autoreload
%autoreload 2
from dpe import DPE
from lib_data import *
import matplotlib.pyplot as plt
import numpy as np
from lib_nn_dpe import NN_dpe
from IPython import display

import serial
%config InlineBackend.figure_formats = ['svg']

import matplotlib
matplotlib.rcParams['font.sans-serif'] = "Arial"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [61]:
dpe = DPE('COM10')
dpe.set_clock(50)
dpe.shape

DAC initialized to a span from -10 V to 10 V
Setting vrefs to default
Setting ADC_CK freq=50000.0kHz
Setting CK_ARRAY freq=50000.0kHz


[64, 64]

In [67]:
g = dpe.read(2,Tdly=1000, method='slow')
plt.imshow(g * 1e6)
plt.colorbar()

....

In [68]:
plt.imshow(g * 1e6, clim=[0,100])
plt.colorbar()

In [70]:
# NOW TRY TAKING THE DATA ONLY ONE COLUMN AT A TIME TO SEE IF WE GET BETTER ACCURACY
from lib_data import *
import scipy.io as sio
mat_contents = sio.loadmat('Exported60Node_GraphNum0.mat')
CMat = mat_contents['A']

CMat[0,33] = 0
CMat[33,0] = 0
#CMat[56,6] = 0
#CMat[6,56] = 0

import dut_func as dut
import time
import struct
import matplotlib
import matplotlib.pyplot as plt

from dpe import DPE
from lib_data import *
from lib_nn_dpe import NN_dpe
from IPython import display

arr = 2
dpe.N_BIT = 1

numTests = 500
appliedVector1 = np.zeros((64,numTests))
appliedVector2 = np.zeros((64,numTests))
randomVector = np.random.randint(2, size=(60,numTests))

appliedVector1[0:60,:] = randomVector[:,:]
appliedVector2[0:60,:] = 1-randomVector[:,:]
neuronVector = appliedVector1[0:60,:] - appliedVector2[0:60,:]

output1 = np.zeros((numTests,60))
output2 = np.zeros((numTests,60))
for c in range(60):
    #output1[:,c]  = dpe.multiply_w_delay(arr, appliedVector1, c_sel=[c,c+1], mode=1, debug=False, delay=5).T
    output1[:,c]  = dpe.multiply(arr, appliedVector1, c_sel=[c,c+1], mode=1, debug=False, Tdly=500).T
    #output2[:,c]  = dpe.multiply_w_delay(arr, appliedVector2, c_sel=[c,c+1], mode=1, debug=False, delay=5).T
    output2[:,c]  = dpe.multiply(arr, appliedVector2, c_sel=[c,c+1], mode=1, debug=False, Tdly=500).T
all_output = np.concatenate((output1, output2))

#SW_output = neuronVector * CMat
#SW_neuron = np.matmul(neuronVector, CMat)
SW_output1 = (CMat.T @ appliedVector1[0:60,:]).T 
SW_output2 = (CMat.T @ appliedVector2[0:60,:]).T 
all_SW_output = np.concatenate((SW_output1, SW_output2))

lin_corrs = []
for c in range(60):
    lin_corrs.append( np.polyfit(all_output[:,c], all_SW_output[:,c], 1))

output_corr  = dpe.lin_corr(all_output, lin_corrs)
output_corr1 = dpe.lin_corr(output1, lin_corrs)
output_corr2 = dpe.lin_corr(output2, lin_corrs)
exp_neuron_output = output_corr1 - output_corr2
SW_neuron_output = SW_output1 - SW_output2

error = output_corr - all_SW_output
fig, ax = plt.subplots()
for c in range(60):
    x = all_SW_output[:,c]
    y = output_corr[:,c]    
    ax.plot(x,y,'.')
zz = np.linspace(0,30,30)  
ax.plot(zz,zz,'-')

fig, ax = plt.subplots()
ax.hist(error.flatten(),bins=60);

fig, ax = plt.subplots()
for c in range(60):
    x = SW_neuron_output[:,c]
    y = exp_neuron_output[:,c]    
    ax.plot(x,y,'.')   
zz = np.linspace(-25,25,50)    
ax.plot(zz,zz,'-')  

fig, ax = plt.subplots()
err_neuron = exp_neuron_output.flatten() - SW_neuron_output.flatten()
ax.hist(err_neuron.flatten(),bins=60);

In [71]:
save_workspace({'lin_corrs': lin_corrs}, note='memHNN_LinearCorrectionsChip42_Prober2System')

Saving lin_corrs(<class 'list'>), to 20200131-115824-memHNN_LinearCorrectionsChip42_Prober2System.pkl
Saving lin_corrs(<class 'list'>), to 20200131-115824-memHNN_LinearCorrectionsChip42_Prober2System.mat


In [74]:
# Find the Linear Correction factors for 10 columns at a time mode
from lib_data import *
import scipy.io as sio
mat_contents = sio.loadmat('Exported60Node_GraphNum0.mat')
CMat = mat_contents['A']

CMat[0,33] = 0
CMat[33,0] = 0
#CMat[56,6] = 0
#CMat[6,56] = 0

import dut_func as dut
import time
import struct
import matplotlib
import matplotlib.pyplot as plt

from dpe import DPE
from lib_data import *
from lib_nn_dpe import NN_dpe
from IPython import display

arr = 2
dpe.N_BIT = 1

numTests = 500
appliedVector1 = np.zeros((64,numTests))
appliedVector2 = np.zeros((64,numTests))
randomVector = np.random.randint(2, size=(60,numTests))

appliedVector1[0:60,:] = randomVector[:,:]
appliedVector2[0:60,:] = 1-randomVector[:,:]
neuronVector = appliedVector1[0:60,:] - appliedVector2[0:60,:]

output1 = np.zeros((numTests,60))
output2 = np.zeros((numTests,60))

#for c in range(60):
#    output1[:,c]  = dpe.multiply(arr, appliedVector1, c_sel=[c,c+1], mode=1, debug=False, Tdly=500).T
#    output2[:,c]  = dpe.multiply(arr, appliedVector2, c_sel=[c,c+1], mode=1, debug=False, Tdly=500).T
#all_output = np.concatenate((output1, output2))

#### Modified version with batching of 10
sizeBatch = 10
numBatches = 6
for ii in range(numBatches):
            colStart = np.around(ii*sizeBatch).astype(int)
            colEnd = np.around((ii+1)*sizeBatch).astype(int)
            output1[:,colStart:colEnd] = dpe.multiply(arr, appliedVector1, c_sel=[colStart, colEnd], mode=1, debug=False, Tdly=500)
            output2[:,colStart:colEnd] = dpe.multiply(arr, appliedVector2, c_sel=[colStart, colEnd], mode=1, debug=False, Tdly=500)
all_output = np.concatenate((output1, output2))            
           

SW_output1 = (CMat.T @ appliedVector1[0:60,:]).T 
SW_output2 = (CMat.T @ appliedVector2[0:60,:]).T 
all_SW_output = np.concatenate((SW_output1, SW_output2))

lin_corrs10AtATime = []
for c in range(60):
    lin_corrs10AtATime.append( np.polyfit(all_output[:,c], all_SW_output[:,c], 1))

output_corr  = dpe.lin_corr(all_output, lin_corrs10AtATime)
output_corr1 = dpe.lin_corr(output1, lin_corrs10AtATime)
output_corr2 = dpe.lin_corr(output2, lin_corrs10AtATime)
exp_neuron_output = output_corr1 - output_corr2
SW_neuron_output = SW_output1 - SW_output2

error = output_corr - all_SW_output
fig, ax = plt.subplots()
for c in range(60):
    x = all_SW_output[:,c]
    y = output_corr[:,c]    
    ax.plot(x,y,'.')
zz = np.linspace(0,30,30)  
ax.plot(zz,zz,'-')

fig, ax = plt.subplots()
ax.hist(error.flatten(),bins=60);

fig, ax = plt.subplots()
for c in range(60):
    x = SW_neuron_output[:,c]
    y = exp_neuron_output[:,c]    
    ax.plot(x,y,'.')   
zz = np.linspace(-25,25,50)    
ax.plot(zz,zz,'-')  

fig, ax = plt.subplots()
err_neuron = exp_neuron_output.flatten() - SW_neuron_output.flatten()
ax.hist(err_neuron.flatten(),bins=60);

In [75]:
save_workspace({'lin_corrs10AtATime': lin_corrs10AtATime}, note='memHNN_LinearCorrections10AtATimeChip42_Prober2System')

Saving lin_corrs10AtATime(<class 'list'>), to 20200131-125613-memHNN_LinearCorrections10AtATimeChip42_Prober2System.pkl
Saving lin_corrs10AtATime(<class 'list'>), to 20200131-125613-memHNN_LinearCorrections10AtATimeChip42_Prober2System.mat


In [88]:
print(np.shape(output1))
print(np.shape(lin_corrs10AtATime))
aa = lin_corrs10AtATime[0:5]
print(np.shape(aa))

(500, 60)
(60, 2)
(5, 2)


In [90]:
# Find optimum Schmidt values when running in 10 columns at a time for approximately 10 cycles

arr = 2
numCycles = 10
numTrials = 2
sizeBatch = 10
numBatches = 6

#startSchmidtVals = [-2, -1.75, -1.5, -1.25,  -1.0, -0.75, -0.5, -0.25, 0, +0.25, +0.5, +1.0, -4.5]
#endSchmidtVals = [-2, -1.75, -1.5, -1.25,  -1.0, -0.75, -0.5, -0.25, 0, +0.25, +0.5, +1.0, +1.4]
startSchmidtVals = [-2.5, -2.0]
endSchmidtVals   = [+0.4, +1.0]
numSchmidt = np.size(startSchmidtVals)

SchmidtCycleVector = np.zeros((numCycles,numSchmidt))

for ss in np.arange(numSchmidt):
    SchmidtCycleVector[:,ss] = np.linspace(startSchmidtVals[ss], endSchmidtVals[ss], numCycles)
    
threshold = 0
noise = 0
numCols = 60

energyHistory = np.zeros((numCycles*numCols+1, numSchmidt, numTrials))
appliedVector1 = np.zeros((64,numSchmidt))
appliedVector2 = np.zeros((64,numSchmidt))
output1 = np.zeros((numSchmidt,sizeBatch))
output2 = np.zeros((numSchmidt,sizeBatch))

neuronVectorHistory = np.zeros((60,numSchmidt, numCycles*numCols+1, numTrials))
neuronVector = np.zeros((60,numSchmidt))
columnUpdateHistory = np.zeros((1,numCycles*numCols+1, numTrials))

for tt in np.arange(numTrials):
    print('Trial number', tt)
    randomVector = np.random.randint(2, size=(60,1))
    initVector = randomVector

    for ss in np.arange(numSchmidt):
        appliedVector1[0:60,ss] = initVector[:,0]
        appliedVector2[0:60,ss] = 1-initVector[:,0]
        neuronVector[0:60,ss] = appliedVector1[0:60,ss] - appliedVector2[0:60,ss]
        neuronVectorHistory[:,ss,0,tt] = appliedVector1[0:60,ss] - appliedVector2[0:60,ss]
        
    #energyHistory[0,tt]= 0.5*np.dot(neuronVector.T, (CMat @ neuronVector))            

    for cc in np.arange(numCycles):
        #randOrderColumns = np.arange(60)
        #np.random.shuffle(randOrderColumns)
        randOrderColumnBatches = np.arange(numBatches).astype(int)
        np.random.shuffle(randOrderColumnBatches)
        
        trackColBatch = 0
        for ii in randOrderColumnBatches:
            colStart = np.around(ii*sizeBatch).astype(int)
            colEnd = np.around((ii+1)*sizeBatch).astype(int)
            #output1[:,colStart:colEnd] = dpe.multiply(arr, appliedVector1, c_sel=[colStart, colEnd], mode=1, debug=False, Tdly=500)
            #output2[:,colStart:colEnd] = dpe.multiply(arr, appliedVector2, c_sel=[colStart, colEnd], mode=1, debug=False, Tdly=500)
            output1[:,:] = dpe.multiply(arr, appliedVector1, c_sel=[colStart, colEnd], mode=1, debug=False, Tdly=500)
            output2[:,:] = dpe.multiply(arr, appliedVector2, c_sel=[colStart, colEnd], mode=1, debug=False, Tdly=500)
            output_corr = noise - dpe.lin_corr(output1, lin_corrs10AtATime[colStart:colEnd]) + dpe.lin_corr(output2, lin_corrs10AtATime[colStart:colEnd])            
            for ss in np.arange(numSchmidt):
                threshVector = threshold - SchmidtCycleVector[cc,ss] * neuronVector[ii*sizeBatch:(ii+1)*sizeBatch, ss]
                for columnIndex in np.arange(sizeBatch):
                    actualColumn = ii*sizeBatch + columnIndex
                    if (output_corr[ss, columnIndex] >= threshVector[columnIndex]):
                        appliedVector1[actualColumn, ss] = 1
                        appliedVector2[actualColumn, ss] = 0
                        neuronVector[actualColumn, ss] = 1
                    else:
                        appliedVector1[actualColumn, ss] = 0
                        appliedVector2[actualColumn, ss] = 1
                        neuronVector[actualColumn, ss] = -1
                    neuronVectorHistory[:, ss, 60*cc + trackColBatch*sizeBatch + columnIndex + 1,tt] = neuronVector[:, ss]
                    #thisEnergy[ss] = 0.5 * np.dot(neuronVector[:, ss].T, (CMat @ neuronVector[:, ss]))
                    thisEnergy = 0.5*np.dot(neuronVector[:,ss], np.dot(CMat, neuronVector[:,ss]))
                    energyHistory[60*cc + trackColBatch*sizeBatch + columnIndex + 1, ss, tt] = thisEnergy
                    columnUpdateHistory[0, 60*cc + trackColBatch*sizeBatch + columnIndex + 1,tt] = actualColumn
            trackColBatch = trackColBatch + 1
            
save_workspace({'lin_corrs10AtATime': lin_corrs10AtATime, 'energyHistory': energyHistory, 'neuronVectorHistory':neuronVectorHistory, 'columnUpdateHistory': columnUpdateHistory, 'startSchmidtVals': startSchmidtVals, 'endSchmidtVals':endSchmidtVals}, note='Prober1_HNN10cyc_2trials_2Schmitt')

Trial number 0
Trial number 1
Saving In(<class 'list'>), Out(<class 'dict'>), g(<class 'numpy.ndarray'>), numCycles(<class 'int'>), numTrials(<class 'int'>), startSchmidtVal(<class 'float'>), endSchmidtVal(<class 'float'>), mat_contents(<class 'dict'>), CMat(<class 'numpy.ndarray'>), arr(<class 'int'>), SchmidtCycleVector(<class 'numpy.ndarray'>), threshold(<class 'int'>), noise(<class 'int'>), numCols(<class 'int'>), appliedVector1(<class 'numpy.ndarray'>), appliedVector2(<class 'numpy.ndarray'>), neuronVectorHistory(<class 'numpy.ndarray'>), neuronVector(<class 'numpy.ndarray'>), columnUpdateHistory(<class 'numpy.ndarray'>), energyHistory(<class 'numpy.ndarray'>), num_updates(<class 'int'>), time_vector(<class 'numpy.ndarray'>), color_idx_array(<class 'numpy.ndarray'>), color_map(<class 'str'>), trial_index(<class 'int'>), energy_vector(<class 'numpy.ndarray'>), lines(<class 'list'>), thisEnergy(<class 'numpy.float64'>), randomVector(<class 'numpy.ndarray'>), initVector(<class 'numpy

In [95]:
# Find optimum Schmidt values when running in 10 columns at a time for approximately 10 cycles

arr = 2
numCycles = 10
numTrials = 80
sizeBatch = 10
numBatches = 6

#startSchmidtVals = [-2, -1.75, -1.5, -1.25,  -1.0, -0.75, -0.5, -0.25, 0, +0.25, +0.5, +1.0, -4.5]
#endSchmidtVals = [-2, -1.75, -1.5, -1.25,  -1.0, -0.75, -0.5, -0.25, 0, +0.25, +0.5, +1.0, +1.4]
startSchmidtVals = [-4.5, -3.5, -2.5, -2.0, -1.5, -1.0, -0.5, 0]
endSchmidtVals   = [0, 0.5, 1.0, 1.5]
numStartSchmidt = np.size(startSchmidtVals)
numEndSchmidt = np.size(endSchmidtVals)
numSchmidt = numStartSchmidt*numEndSchmidt
SchmidtCycleVector = np.zeros((numCycles,numSchmidt))

indy = 0
for ss in np.arange(numStartSchmidt):
    for ee in np.arange(numEndSchmidt):        
        SchmidtCycleVector[:,indy] = np.linspace(startSchmidtVals[ss], endSchmidtVals[ee], numCycles)
        indy = indy+1
    
threshold = 0
noise = 0
numCols = 60

energyHistory = np.zeros((numCycles*numCols+1, numSchmidt, numTrials))
appliedVector1 = np.zeros((64,numSchmidt))
appliedVector2 = np.zeros((64,numSchmidt))
output1 = np.zeros((numSchmidt,sizeBatch))
output2 = np.zeros((numSchmidt,sizeBatch))

neuronVectorHistory = np.zeros((60,numSchmidt, numCycles*numCols+1, numTrials))
neuronVector = np.zeros((60,numSchmidt))
columnUpdateHistory = np.zeros((1,numCycles*numCols+1, numTrials))

for tt in np.arange(numTrials):
    print('Trial number', tt)
    randomVector = np.random.randint(2, size=(60,1))
    initVector = randomVector

    for ss in np.arange(numSchmidt):
        appliedVector1[0:60,ss] = initVector[:,0]
        appliedVector2[0:60,ss] = 1-initVector[:,0]
        neuronVector[0:60,ss] = appliedVector1[0:60,ss] - appliedVector2[0:60,ss]
        neuronVectorHistory[:,ss,0,tt] = appliedVector1[0:60,ss] - appliedVector2[0:60,ss]
        
    #energyHistory[0,tt]= 0.5*np.dot(neuronVector.T, (CMat @ neuronVector))            

    for cc in np.arange(numCycles):
        #randOrderColumns = np.arange(60)
        #np.random.shuffle(randOrderColumns)
        randOrderColumnBatches = np.arange(numBatches).astype(int)
        np.random.shuffle(randOrderColumnBatches)
        
        trackColBatch = 0
        for ii in randOrderColumnBatches:
            colStart = np.around(ii*sizeBatch).astype(int)
            colEnd = np.around((ii+1)*sizeBatch).astype(int)
            #output1[:,colStart:colEnd] = dpe.multiply(arr, appliedVector1, c_sel=[colStart, colEnd], mode=1, debug=False, Tdly=500)
            #output2[:,colStart:colEnd] = dpe.multiply(arr, appliedVector2, c_sel=[colStart, colEnd], mode=1, debug=False, Tdly=500)
            output1[:,:] = dpe.multiply(arr, appliedVector1, c_sel=[colStart, colEnd], mode=1, debug=False, Tdly=500)
            output2[:,:] = dpe.multiply(arr, appliedVector2, c_sel=[colStart, colEnd], mode=1, debug=False, Tdly=500)
            output_corr = noise - dpe.lin_corr(output1, lin_corrs10AtATime[colStart:colEnd]) + dpe.lin_corr(output2, lin_corrs10AtATime[colStart:colEnd])            
            for ss in np.arange(numSchmidt):
                threshVector = threshold - SchmidtCycleVector[cc,ss] * neuronVector[ii*sizeBatch:(ii+1)*sizeBatch, ss]
                for columnIndex in np.arange(sizeBatch):
                    actualColumn = ii*sizeBatch + columnIndex
                    if (output_corr[ss, columnIndex] >= threshVector[columnIndex]):
                        appliedVector1[actualColumn, ss] = 1
                        appliedVector2[actualColumn, ss] = 0
                        neuronVector[actualColumn, ss] = 1
                    else:
                        appliedVector1[actualColumn, ss] = 0
                        appliedVector2[actualColumn, ss] = 1
                        neuronVector[actualColumn, ss] = -1
                    neuronVectorHistory[:, ss, 60*cc + trackColBatch*sizeBatch + columnIndex + 1,tt] = neuronVector[:, ss]
                    #thisEnergy[ss] = 0.5 * np.dot(neuronVector[:, ss].T, (CMat @ neuronVector[:, ss]))
                    thisEnergy = 0.5*np.dot(neuronVector[:,ss], np.dot(CMat, neuronVector[:,ss]))
                    energyHistory[60*cc + trackColBatch*sizeBatch + columnIndex + 1, ss, tt] = thisEnergy
                    columnUpdateHistory[0, 60*cc + trackColBatch*sizeBatch + columnIndex + 1,tt] = actualColumn
            trackColBatch = trackColBatch + 1
            
save_workspace({'lin_corrs10AtATime': lin_corrs10AtATime, 'energyHistory': energyHistory, 'neuronVectorHistory':neuronVectorHistory, 'columnUpdateHistory': columnUpdateHistory, 'startSchmidtVals': startSchmidtVals, 'endSchmidtVals':endSchmidtVals}, note='Prober1_HNN10cyc_80trials_50Schmitt')

Trial number 0
Trial number 1
Trial number 2
Trial number 3
Trial number 4
Trial number 5
Trial number 6
Trial number 7
Trial number 8
Trial number 9
Trial number 10
Trial number 11
Trial number 12
Trial number 13
Trial number 14
Trial number 15
Trial number 16
Trial number 17
Trial number 18
Trial number 19
Trial number 20
Trial number 21
Trial number 22
Trial number 23
Trial number 24
Trial number 25
Trial number 26
Trial number 27
Trial number 28
Trial number 29
Trial number 30
Trial number 31
Trial number 32
Trial number 33
Trial number 34
Trial number 35
Trial number 36
Trial number 37
Trial number 38
Trial number 39
Trial number 40
Trial number 41
Trial number 42
Trial number 43
Trial number 44
Trial number 45
Trial number 46
Trial number 47
Trial number 48
Trial number 49
Trial number 50
Trial number 51
Trial number 52
Trial number 53
Trial number 54
Trial number 55
Trial number 56
Trial number 57
Trial number 58
Trial number 59
Trial number 60
Trial number 61
Trial number 62
Tr

MemoryError: 

In [101]:
save_workspace({'energyHistory': energyHistory, 'neuronVectorHistory':neuronVectorHistory, 'columnUpdateHistory': columnUpdateHistory, 'startSchmidtVals': startSchmidtVals, 'endSchmidtVals': endSchmidtVals}, note='Prober1_HNN10cyc_80trials_50Schmitt')

Saving energyHistory(<class 'numpy.ndarray'>), neuronVectorHistory(<class 'numpy.ndarray'>), columnUpdateHistory(<class 'numpy.ndarray'>), startSchmidtVals(<class 'list'>), endSchmidtVals(<class 'list'>), to 20200202-084535-Prober1_HNN10cyc_80trials_50Schmitt.pkl


MemoryError: 

In [ ]:
save_workspace({'energyHistory': energyHistory}, note='Prober1_HNN10cyc_80trials_50Schmitt')